In [ ]:
!pip install spark-nlp==3.4.2 pyspark==3.2.0

     |████████████████████████████████| 142 kB 5.3 MB/s 
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 73.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=942043487cc69cb7683279ee42c787975b3271ac7963f84977bf4b3f964516fe
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
!pip install pyspark

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.7.5

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
     |████████████████████████████████| 139 kB 5.1 MB/s 


In [ ]:
import sparknlp
spark = sparknlp.start(spark23=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving news_summary.csv to news_summary.csv


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path

In [ ]:
df = pd.read_csv("news_summary.csv", encoding= "latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df = df[['text','ctext']]

In [ ]:
df= df.dropna()

In [ ]:
df.iloc[0,1]

'The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the Dama

In [ ]:
df.iloc[:,0]

0       The Administration of Union Territory Daman an...
1       Malaika Arora slammed an Instagram user who tr...
2       The Indira Gandhi Institute of Medical Science...
3       Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4       Hotels in Maharashtra will train their staff t...
                              ...                        
4509    Fruit juice concentrate maker Rasna is eyeing ...
4510    Former Indian cricketer Sachin Tendulkar atten...
4511    Aamir Khan, while talking about reality shows ...
4512    The Maharashtra government has initiated an in...
4513    At least 400 languages or more than half langu...
Name: text, Length: 4396, dtype: object

In [ ]:
spark_df = df.iloc[:,0]

In [ ]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

#create or get Spark Session

spark = sparknlp.start()

sparknlp.version()
spark.version

from sparknlp.base import *

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)



In [ ]:
spark_df = sqlContext.createDataFrame(df)

In [ ]:
spark_df

DataFrame[text: string, ctext: string]

In [ ]:
documenter = DocumentAssembler()\
    .setInputCol("ctext")\
    .setOutputCol("document")

In [ ]:
t5 = sparknlp.annotator.T5Transformer.pretrained() \
    .setTask("summarize:") \
    .setInputCols(["document"]) \
    .setOutputCol("summaries")

t5_small download started this may take some time.
Approximate size to download 139 MB
[OK!]


In [ ]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

In [ ]:
doc_df = documenter.transform(spark_df)
doc_df.show()

+--------------------+--------------------+--------------------+
|                text|               ctext|            document|
+--------------------+--------------------+--------------------+
|The Administratio...|The Daman and Diu...|[[document, 0, 23...|
|Malaika Arora sla...|From her special ...|[[document, 0, 23...|
|The Indira Gandhi...|The Indira Gandhi...|[[document, 0, 21...|
|Lashkar-e-Taiba's...|Lashkar-e-Taiba's...|[[document, 0, 23...|
|Hotels in Maharas...|Hotels in Mumbai ...|[[document, 0, 32...|
|A 32-year-old man...|An alleged suspec...|[[document, 0, 22...|
|The Delhi High Co...|In an interesting...|[[document, 0, 23...|
|A 60-year old Dal...| A 60-year old Da...|[[document, 0, 35...|
|An inquiry by the...|Two years after a...|[[document, 0, 14...|
|The Congress part...|It sounds like sa...|[[document, 0, 15...|
|India's food regu...|The Food Safety a...|[[document, 0, 21...|
|The mother of Har...|Bharti Sharma, mo...|[[document, 0, 26...|
|Municipal Corpora...|Onl

In [ ]:
pipeline = Pipeline().setStages([documenter, t5])
results = pipeline.fit(spark_df).transform(spark_df)

In [ ]:
results.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|               ctext|            document|           summaries|
+--------------------+--------------------+--------------------+--------------------+
|The Administratio...|The Daman and Diu...|[[document, 0, 23...|[[document, 0, 32...|
|Malaika Arora sla...|From her special ...|[[document, 0, 23...|[[document, 0, 23...|
|The Indira Gandhi...|The Indira Gandhi...|[[document, 0, 21...|[[document, 0, 24...|
|Lashkar-e-Taiba's...|Lashkar-e-Taiba's...|[[document, 0, 23...|[[document, 0, 21...|
|Hotels in Maharas...|Hotels in Mumbai ...|[[document, 0, 32...|[[document, 0, 17...|
|A 32-year-old man...|An alleged suspec...|[[document, 0, 22...|[[document, 0, 15...|
|The Delhi High Co...|In an interesting...|[[document, 0, 23...|[[document, 0, 25...|
|A 60-year old Dal...| A 60-year old Da...|[[document, 0, 35...|[[document, 0, 19...|
|An inquiry by the...|Two years after a...|[[document,

In [ ]:
results.select(["ctext","summaries.result"]).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
type(results)

pyspark.sql.dataframe.DataFrame

In [ ]:
# df3 = sqlContext.createDataFrame(results.head(1), results.schema)

In [ ]:
reference= "The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media."
candidate= "the order made it compulsory for women to tie rakhis to male colleagues on the occasion of Rakshabandhan on august 7 . the administration was forced to withdraw the decision within 24 hours of issuing the circular ."

In [ ]:
pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rouge3','rougeL'], use_stemmer = True)
scores = scorer.score(reference,
                      candidate)

In [ ]:
scores

{'rouge1': Score(precision=1.0, recall=0.6, fmeasure=0.7499999999999999),
 'rouge2': Score(precision=0.9142857142857143, recall=0.5423728813559322, fmeasure=0.6808510638297872),
 'rouge3': Score(precision=0.8823529411764706, recall=0.5172413793103449, fmeasure=0.6521739130434783),
 'rougeL': Score(precision=1.0, recall=0.6, fmeasure=0.7499999999999999)}

Analysis of Performance on different categories of Articles

In [ ]:
df.dropna(inplace = True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving top_15_articles.csv to top_15_articles.csv


In [ ]:
new_df = pd.read_csv("top_15_articles.csv", encoding= "latin-1")

In [ ]:
new_df.shape

(15, 2)

In [ ]:
#new_df.iloc[:,0]
spark_df_1 = new_df.iloc[:,-1]
spark_df_1 

0     Akshay Kumar, whose forthcoming movie Toilet E...
1     2017 is turning out to be an important year fo...
2     Actor Emraan Hashmi has been in the Hindi film...
3     Although actor Kriti Sanon is just three films...
4     Shah Rukh Khan is not called the ?King of Roma...
5     Pakistan Cricket Board (PCB) has reserved arou...
6     Tennis star Sania Mirza recently lauded the In...
7     Legendary Australian fast bowler Glenn Mcgrath...
8     Manchester City have taken their summer spendi...
9     The Indian women's cricket team is the toast o...
10    President Donald Trump?s son-in-law says the T...
11    A Republican senator said Tuesday that US Pres...
12    BJP chief Amit Shah on Monday said he is happy...
13    Less than a day after resigning as Bihar chief...
14    As Nitish Kumar meets his party - Janata Dal U...
Name: ctext, dtype: object

In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

spark_df_1 = sqlContext.createDataFrame(new_df)
spark_df_1

DataFrame[text: string, ctext: string]

In [ ]:
# documenter = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

# t5 = sparknlp.annotator.T5Transformer.pretrained() \
#     .setTask("summarize:") \
#     .setInputCols(["document"]) \
#     .setOutputCol("summaries")

new_doc_df = documenter.transform(spark_df_1)
new_doc_df.show()

+--------------------+--------------------+--------------------+
|                text|               ctext|            document|
+--------------------+--------------------+--------------------+
|Actor Akshay Kuma...|Akshay Kumar, who...|[[document, 0, 25...|
|Actor Rana Daggub...|2017 is turning o...|[[document, 0, 11...|
|Actor Emraan Hash...|Actor Emraan Hash...|[[document, 0, 17...|
|Actress Kriti San...|Although actor Kr...|[[document, 0, 18...|
|Actor Shah Rukh K...|Shah Rukh Khan is...|[[document, 0, 18...|
|Pakistan Cricket ...|Pakistan Cricket ...|[[document, 0, 38...|
|Sania Mirza has c...|Tennis star Sania...|[[document, 0, 74...|
|Ex-Australian pac...|Legendary Austral...|[[document, 0, 83...|
|Premier League cl...|Manchester City h...|[[document, 0, 31...|
|Gautam Gambhir, w...|The Indian women'...|[[document, 0, 15...|
|US President Dona...|President Donald ...|[[document, 0, 88...|
|Republican Senato...|A Republican sena...|[[document, 0, 14...|
|Bharatiya Janata ...|BJP

In [ ]:
documenter1 = DocumentAssembler()\
    .setInputCol("ctext")\
    .setOutputCol("document")

In [ ]:
pipeline = Pipeline().setStages([documenter1, t5])
new_results = pipeline.fit(spark_df_1).transform(spark_df_1)
new_results.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|               ctext|            document|           summaries|
+--------------------+--------------------+--------------------+--------------------+
|Actor Akshay Kuma...|Akshay Kumar, who...|[[document, 0, 25...|[[document, 0, 16...|
|Actor Rana Daggub...|2017 is turning o...|[[document, 0, 11...|[[document, 0, 14...|
|Actor Emraan Hash...|Actor Emraan Hash...|[[document, 0, 17...|[[document, 0, 18...|
|Actress Kriti San...|Although actor Kr...|[[document, 0, 18...|[[document, 0, 15...|
|Actor Shah Rukh K...|Shah Rukh Khan is...|[[document, 0, 18...|[[document, 0, 23...|
|Pakistan Cricket ...|Pakistan Cricket ...|[[document, 0, 38...|[[document, 0, 24...|
|Sania Mirza has c...|Tennis star Sania...|[[document, 0, 74...|[[document, 0, 22...|
|Ex-Australian pac...|Legendary Austral...|[[document, 0, 83...|[[document, 0, 27...|
|Premier League cl...|Manchester City h...|[[document,

In [ ]:
new_results= new_results.select(["text","summaries.result"])

In [ ]:
hyp_list= new_results.select('text').rdd.flatMap(lambda x: x).collect()


In [ ]:
# importing the native rouge library
from rouge_score import rouge_scorer

# a list of the hypothesis documents
hyp = ['This is the first sample', 'This is another example']
# a list of the references documents
ref = ['This is the first sentence', 'It is one more sentence']

# make a RougeScorer object with rouge_types=['rouge1']
scorer = rouge_scorer.RougeScorer(['rouge1'])

# a dictionary that will contain the results
results = {'precision': [], 'recall': [], 'fmeasure': []}

# for each of the hypothesis and reference documents pair
for (h, r) in zip(hyp, ref):
    # computing the ROUGE
    score = scorer.score(h, r)
    # separating the measurements
    precision, recall, fmeasure = score['rouge1']
    # add them to the proper list in the dictionary
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['fmeasure'].append(fmeasure)

In [ ]:
new_results.head()

Row(text='Actor Akshay Kumar has said that talking about toilets is still considered a taboo in India. He added, "It\'s also about the mindset. In places where I\'ve gone, I heard people saying, \'We don\'t do it (defecate) in the same house where we cook food\'. But how can they openly defecate on the field, from where the food is coming?"', result=['Akshay Kumar says the film is a "taboo topic in Indian society" he says the film is a way to change people\'s mindset . the movie is directed by Shree Narayan Singh .'])